<a href="https://colab.research.google.com/github/Jarvis-BITS/midodrine-mimic-iv/blob/main/notebook/01_1_cohort_extraction_midodrine_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [1]:
from google.cloud import bigquery
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(rc={'figure.figsize':(16,10)}, font_scale=1.3)
import sys
import warnings
warnings.filterwarnings('ignore')

Authenticate with GCP account having MIMIC-IV dataset

In [2]:
from google.colab import auth

auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
%load_ext google.colab.data_table

In [4]:
%load_ext google.cloud.bigquery

In [5]:
project_id = "my-project-eicu" #@param {type:"string"} #Enter your project ID from GCP bigquey here
bqclient = bigquery.Client(project=project_id)

# BigQuery data extraction of patient details in ICU (1st day)

In [6]:
population_string = """
WITH patient_details AS (
  SELECT
  ie.subject_id, ie.hadm_id, ie.stay_id
  , pat.gender, ie.ethnicity
  , ie.admittime, ie.dischtime
  , DATETIME_DIFF(ie.admittime, DATETIME(pat.anchor_year, 1, 1, 0, 0, 0), YEAR) + pat.anchor_age AS age
  , adm.deathtime
  , ie.hospital_expire_flag

-- icu level factors
  , ie.icu_intime, ie.icu_outtime
  , ie.first_icu_stay
  , ie.los_icu, ie.los_hospital
  FROM `physionet-data.mimic_derived.icustay_detail` ie
  INNER JOIN `physionet-data.mimic_core.admissions` adm
  ON ie.hadm_id = adm.hadm_id
  INNER JOIN `physionet-data.mimic_core.patients` pat
  ON ie.subject_id = pat.subject_id

-- midodrine cohort
  )
  , midodrine AS (
    SELECT hadm_id, pharmacy_id, starttime AS drug_starttime
    FROM `physionet-data.mimic_hosp.pharmacy`
    WHERE lower(medication) like 'midodrine'
  )
  , midodrine_exclude AS (
    SELECT stay_id,
    CASE WHEN LOGICAL_AND(drug_starttime < DATE_TRUNC(intime - interval '1' day, DAY) OR drug_starttime > outtime) THEN 1 
    ELSE 0 END AS midodrine_exclude
    FROM `physionet-data.mimic_icu.icustays` 
    LEFT JOIN midodrine USING (hadm_id)
    GROUP BY stay_id
  )
  , midodrine_include AS (
    SELECT stay_id, mi.pharmacy_id,
    CASE WHEN LOGICAL_AND(drug_starttime is NULL)
    OR LOGICAL_OR(drug_starttime BETWEEN DATE_TRUNC(intime - interval '1' day, DAY) AND outtime) THEN 1
    ELSE 0 END AS midodrine_include
    FROM `physionet-data.mimic_icu.icustays`  
    LEFT JOIN midodrine mi USING (hadm_id)
    GROUP BY stay_id, mi.pharmacy_id
  )
  , midodrine_first AS (
  SELECT stay_id, min(drug_starttime) AS drug_starttime 
  FROM `physionet-data.mimic_icu.icustays` LEFT JOIN midodrine mi USING (hadm_id)
  WHERE drug_starttime BETWEEN DATE_TRUNC(intime - interval '1' day, DAY) AND outtime
  GROUP BY stay_id
  )
 , midodrine_dose AS (
  SELECT ic.stay_id, AVG(doses_per_24_hrs) AS avg_doses_per_24_hrs
  FROM `physionet-data.mimic_hosp.pharmacy` ph
  INNER JOIN `physionet-data.mimic_icu.icustays`ic
  ON ph.hadm_id = ic.hadm_id
  GROUP BY ic.stay_id
  )

 , sepsis3 AS (
   SELECT stay_id, sepsis3
   FROM `physionet-data.mimic_derived.sepsis3`
 )

  , adm_weekday AS (
    SELECT stay_id,
      EXTRACT(DAYOFWEEK FROM intime) AS adm_weekday
    FROM `physionet-data.mimic_icu.icustays`
  )

  , population AS (
    SELECT *  
    FROM (SELECT DISTINCT stay_id, first_careunit, FROM `physionet-data.mimic_icu.icustays`) a
    LEFT JOIN patient_details USING (stay_id)
    LEFT JOIN midodrine_first USING (stay_id)
    LEFT JOIN midodrine_exclude USING (stay_id)
    LEFT JOIN midodrine_include USING (stay_id)
    LEFT JOIN midodrine_dose USING (stay_id)
    LEFT JOIN sepsis3 USING (stay_id)
    LEFT JOIN adm_weekday USING (stay_id)
    )
    SELECT * FROM population
"""

In [14]:
mortality_string = """
SELECT ic.stay_id,
MAX(CASE WHEN pa.dod is not null THEN 1 ELSE 0 END) AS mortality,
MAX(CASE WHEN pa.dod <= DATETIME_ADD(ic.intime , INTERVAL '28' DAY) AND pa.dod >= ic.intime THEN 1 ELSE 0 END) AS mort_28_day
FROM `physionet-data.mimic_icu.icustays` ic
LEFT JOIN `physionet-data.mimic_core.patients` pa
ON ic.subject_id = pa.subject_id
GROUP BY ic.stay_id
"""

# Building cohort table

## Extract population info

In [15]:
population_data = (
    bqclient.query(population_string)
    .result()
    .to_dataframe()
)

In [ ]:
population_data.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76540 entries, 0 to 76539
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   stay_id               76540 non-null  int64         
 1   first_careunit        76540 non-null  object        
 2   subject_id            76540 non-null  int64         
 3   hadm_id               76540 non-null  int64         
 4   gender                76540 non-null  object        
 5   ethnicity             76540 non-null  object        
 6   admittime             76540 non-null  datetime64[ns]
 7   dischtime             76540 non-null  datetime64[ns]
 8   age                   76540 non-null  int64         
 9   deathtime             8813 non-null   datetime64[ns]
 10  hospital_expire_flag  76540 non-null  int64         
 11  icu_intime            76540 non-null  datetime64[ns]
 12  icu_outtime           76540 non-null  datetime64[ns]
 13  first_icu_stay  

## Building the cohort table (consisting of only ICU patients)
1. Include adults (>= 18 years' old)
2. Include the first ICU admission of each patient
3. Include patients whose ICU care is 'MICU' or 'SICU'
4. Exclude those who accepted midodrine more than one day before icu care or after icu care (have been done before)

Note: midodrine_include is just opposite of midodrine_exclude so use either of the two for this

Filtering on age

In [16]:
cohort_1 = population_data.loc[population_data['age'] >= 18]
cohort_1.shape

(83999, 23)

Filtering on first ICU admission

In [17]:
cohort_2 = cohort_1.loc[cohort_1['first_icu_stay']  == True]
cohort_2.shape

(73910, 23)

Filtering out if death/discharge before 24hrs of intime

In [18]:
# 24 hrs = 86400s ;(24x60x60)
cohort_3 = cohort_2.loc[((cohort_2['deathtime'] - cohort_2['icu_intime']).dt.total_seconds() > 86400) | (cohort_2['deathtime'].isnull())] # Passed away after 24hrs or alive
cohort_3 = cohort_3.loc[((cohort_3['icu_outtime'] - cohort_3['icu_intime']).dt.total_seconds() > 86400)] # Discharged in 24hrs 
cohort_3 = cohort_3.loc[((cohort_3['dischtime'] - cohort_3['icu_intime']).dt.total_seconds() > 86400)] # Discharged in 24hrs 
cohort_3.shape

(58497, 23)

 Filtering in only MICU & SICU first care units

In [19]:
cohort_4 = cohort_3.loc[cohort_3['first_careunit'].str.contains("micu|sicu", case=False)]
cohort_4.shape

(39453, 23)

Filtering on sepsis 

In [20]:
cohort_5 = cohort_4.loc[cohort_4['sepsis3']  == True]
cohort_5.shape

(22582, 23)

Filtering on vasopressor treatments

In [21]:
vaso_id = (
    bqclient.query(vasopressor_id)
    .result()
    .to_dataframe()   
)

In [22]:
cohort_6 = pd.merge(cohort_5, vaso_id, on='stay_id', how='inner')
cohort_6.shape

(10928, 23)

Filtering on midodrine ***perscribed*** for first time

In [23]:
cohort_table = cohort_6.loc[cohort_6['midodrine_include'] == 1]
cohort_table.shape

(9792, 23)

## Divide the Midodrine Group

In [40]:
cohort_mid = cohort_table.loc[pd.notnull(cohort_table['drug_starttime'])]
cohort_mid

,stay_id,first_careunit,subject_id,hadm_id,gender,ethnicity,admittime,dischtime,age,deathtime,...,first_icu_stay,los_icu,los_hospital,drug_starttime,midodrine_exclude,pharmacy_id,midodrine_include,avg_doses_per_24_hrs,sepsis3,adm_weekday
7,36014877,Trauma SICU (TSICU),11861017,27162817,M,WHITE,2189-03-15 15:47:00,2189-05-06 17:00:00,87,NaT,...,True,46.42,52,2189-04-17 16:00:00,0,45772957.0,1,1.595745,True,1
8,36014877,Trauma SICU (TSICU),11861017,27162817,M,WHITE,2189-03-15 15:47:00,2189-05-06 17:00:00,87,NaT,...,True,46.42,52,2189-04-17 16:00:00,0,82401425.0,1,1.595745,True,1
9,36014877,Trauma SICU (TSICU),11861017,27162817,M,WHITE,2189-03-15 15:47:00,2189-05-06 17:00:00,87,NaT,...,True,46.42,52,2189-04-17 16:00:00,0,55309217.0,1,1.595745,True,1
10,36014877,Trauma SICU (TSICU),11861017,27162817,M,WHITE,2189-03-15 15:47:00,2189-05-06 17:00:00,87,NaT,...,True,46.42,52,2189-04-17 16:00:00,0,26557617.0,1,1.595745,True,1
11,36014877,Trauma SICU (TSICU),11861017,27162817,M,WHITE,2189-03-15 15:47:00,2189-05-06 17:00:00,87,NaT,...,True,46.42,52,2189-04-17 16:00:00,0,67458919.0,1,1.595745,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10899,38743934,Medical/Surgical Intensive Care Unit (MICU/SICU),19894790,27933693,M,WHITE,2140-08-25 17:58:00,2140-09-13 18:00:00,84,2140-09-13 18:00:00,...,True,16.88,19,2140-09-01 14:00:00,0,23223011.0,1,1.947368,True,5
10900,38743934,Medical/Surgical Intensive Care Unit (MICU/SICU),19894790,27933693,M,WHITE,2140-08-25 17:58:00,2140-09-13 18:00:00,84,2140-09-13 18:00:00,...,True,16.88,19,2140-09-01 14:00:00,0,19002419.0,1,1.947368,True,5
10919,39446578,Medical/Surgical Intensive Care Unit (MICU/SICU),14781720,23795457,F,WHITE,2189-01-07 21:03:00,2189-01-26 15:11:00,92,NaT,...,True,4.08,19,2189-01-26 14:00:00,0,22076632.0,1,1.409091,True,5
10926,39918058,Medical/Surgical Intensive Care Unit (MICU/SICU),15689523,23914765,M,ASIAN,2159-06-25 16:25:00,2159-07-24 19:30:00,67,NaT,...,True,29.17,29,2159-07-08 14:00:00,0,79916784.0,1,1.453846,True,2


# **Insert Part**: 
check if there's any missing data for the midodrine group

1. have a look at all the pharmacy id used in the midodrine group

In [ ]:
list(cohort_mid.pharmacy_id.unique())

2. extract pharmacy id from midodrine group, mimic_hosp.emar_detail, mimic_hosp.prescriptions

In [53]:
# these are search strategy. Since it's too slow to run the following code in colab,
# the code was executed in GCP directly and the data can be found in GitHub 'check_data_midodrine' folder
emar_string = """
SELECT subject_id, pharmacy_id
FROM physionet-data.mimic_hosp.emar_detail
WHERE pharmacy_id is not null
"""

prescriptions_string = """
SELECT subject_id, pharmacy_id
FROM physionet-data.mimic_hosp.prescriptions
WHERE lower(drug) like 'midodrine'
"""

3. connect to Google Drive to extract the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

4. import data from google drive

In [58]:
# emar_data = pd.read_csv('/content/drive/MyDrive/bq-results-20220709-073542-1657352178548/check_data_emar_detail.csv')
# emar_data.shape

(26711806, 2)

In [60]:
prescriptions_data = pd.read_csv('/content/drive/MyDrive/bq-results-20220709-073718-1657352391947/check_data_prescriptions.csv')
prescriptions_data.shape

(12795, 2)

5. extract those who have been to the ICU

In [64]:
icu_subject_id = cohort_6.subject_id # notice, here the cohort_6, not the populations_data table, is used

In [65]:
# icu_emar = emar_data[emar_data['subject_id'].isin(icu_subject_id)]
icu_prescriptions = prescriptions_data[prescriptions_data['subject_id'].isin(icu_subject_id)]

In [66]:
# print(icu_emar.shape)
print(icu_prescriptions.shape)

(5245580, 2)
(5604, 2)


6. Since there is no way to tell which pharmacy id in emar_detail stands for midodrine, the following part only use prescriptions_data

In [78]:
# check if there is any missing midodrine data by pharmacy_id
print('-----First-------')
print(icu_prescriptions.pharmacy_id.unique().shape[0])
print(cohort_mid.pharmacy_id.unique().shape[0])

print('-----Second-------')
print(icu_prescriptions.subject_id.unique().shape[0])
print(cohort_mid.subject_id.unique().shape[0])

# to make sure that previous operation has made no mistaks (check if unique(stay_id) == 851, as previous cohort data shows)
print('-----Third-------')
print(cohort_mid.stay_id.unique().shape[0])

# check if all midodrine from pharmacy module is contained in prescriptions module
print('-----Fourth-------')
print(cohort_mid[cohort_mid['pharmacy_id'].isin(icu_prescriptions.pharmacy_id)].pharmacy_id.unique().shape[0])
print(cohort_mid[cohort_mid['subject_id'].isin(icu_prescriptions.subject_id)].subject_id.unique().shape[0])

-----First-------
5604
1969
-----Second-------
1096
775
-----Third-------
851
-----Fourth-------
1969
775


It seems that there is indeed some missing data, but we need to further check if the extra data comes from those who were given midodrine a day befor ICU admission or after the ICU admission

7. check using pharmacy id from cohort_6 to see if the set of pharmacy_id from cohort_6 contains the set from prescriptions

In [80]:
print('-----First-----')
print(cohort_6.pharmacy_id.unique().shape[0])

print('-----Second-----')
print(icu_prescriptions[icu_prescriptions['pharmacy_id'].isin(cohort_6.pharmacy_id)].pharmacy_id.unique().shape[0])

-----First-----
3106
-----Second-----
3105


There is a huge gap between 5604 and 3105, showing that there is indeed some missing midodrine data which is in the prescriptions